### Setup

In [1]:
%run -i '../scripts/tf_test.py'

In [2]:
import tensorflow as tf
tf.enable_eager_execution()

from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.__version__

'1.13.1'

In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9557547591947972776
]


## Start program

### Select a model

In [5]:
'''
Model 
'''

img_size = [224,224]

mobile_net = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False)
mobile_net.trainable=False

model = tf.keras.Sequential([
    mobile_net,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [29]:
'''
Model 
'''

img_size = [224,224]

app = tf.keras.applications.ResNet50(input_shape=(224, 224, 3))
app.trainable=False

model = tf.keras.Sequential([
  app,
#   tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(2, activation = 'sigmoid')
])

102858752/102853048 [==============================] - 11s 0us/step


In [42]:
'''
Model 
'''

img_size = [320,320]

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,kernel_size=(3,3), strides=(1, 1),activation='relu',input_shape=(320,320,3)),
#     tf.keras.layers.Dense(batch_size, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation=tf.nn.sigmoid)
])

In [21]:
'''
Model 
'''

img_size = [320,320]

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(320,320,3)),
    tf.keras.layers.Dense(2, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation=tf.nn.sigmoid)
])

### Load Data

In [6]:
'''
Set Params

sample_size: 0-1
'''

sample_size = 1.0

batch_size = 64

In [7]:
'''
Load CSVs
'''

df = GetCSVs(sample_size)
print(df['Catenary'].value_counts())

img_paths,labels = GetPaths(df)

[Errno 2] File b'../data/output_images/China/CHN.csv' does not exist: b'../data/output_images/China/CHN.csv'
1    320
0    320
Name: Catenary, dtype: int64
640


In [8]:
'''
Split, shuffle, and batch images into train/test datasets
'''

train_image_label_ds, test_image_label_ds = SplitDataSet(img_paths, labels)

train_ds = ShuffleBatch(train_image_label_ds,len(img_paths),BATCH_SIZE = batch_size) 
test_ds = ShuffleBatch(test_image_label_ds,len(img_paths),BATCH_SIZE = batch_size)

(224, 224, 3)
(224, 224, 3)


### Compile, Fit, and Evaluate

In [9]:
model.compile(optimizer='rmsprop',
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [38]:
steps_per_epoch = int(tf.ceil(len(img_paths)/batch_size).numpy())

model.fit(train_ds,steps_per_epoch=steps_per_epoch, epochs=10)

model.evaluate(test_ds,steps=steps_per_epoch)

Epoch 1/10
5/5 [==============================] - 41s 8s/step - loss: 0.7488 - acc: 0.5656
Epoch 2/10
5/5 [==============================] - 32s 6s/step - loss: 0.6867 - acc: 0.5938
Epoch 3/10
5/5 [==============================] - 32s 6s/step - loss: 0.6036 - acc: 0.6891
Epoch 4/10
5/5 [==============================] - 32s 6s/step - loss: 0.5870 - acc: 0.6719
Epoch 5/10
5/5 [==============================] - 32s 6s/step - loss: 0.5363 - acc: 0.7250
Epoch 6/10
5/5 [==============================] - 32s 6s/step - loss: 0.4927 - acc: 0.7469
Epoch 7/10
5/5 [==============================] - 32s 6s/step - loss: 0.4836 - acc: 0.7719
Epoch 8/10
5/5 [==============================] - 32s 6s/step - loss: 0.4648 - acc: 0.7844
Epoch 9/10
5/5 [==============================] - 32s 6s/step - loss: 0.4251 - acc: 0.8156
Epoch 10/10
5/5 [==============================] - 40s 8s/step - loss: 0.8573 - acc: 0.6953


[0.8572579145431518, 0.6953125]